In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import networkx as nx

In [2]:
input_ = """#E######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#""".split("\n")

In [28]:
with open("input.txt", "r") as f:
    input_ = f.read().split("\n")[:-1]

In [3]:
# Dimensions of the cave
row_n = len(input_)-2
column_n = len(input_[0])-2
row_n, column_n

(4, 6)

In [4]:
# First I create all the nodes
nodes = []
for row in range(row_n):
    for column in range(column_n):
        node = (row, column)
        nodes.append(node)

In [5]:
# Then I get the wind directions and inital placement
wind_dict = {"<":[],">":[], "^":[],"v":[]}
for i, row in enumerate(input_[1:-1]):
    for j, column in enumerate(row[1:-1]):
        if column != ".":
             wind_dict[column].append((i,j)) 

In [32]:
20*35

700

In [6]:
# Because of math, I know that the wind will be back at the inital state after 12 min for the test case 
# (6 and 4 both divides 12) (for actual input this is 700)
# I will therefor calculate the placement of winds after 1 min to and including 11 min. Then I will be able to find
# the currently unoccupied nodes for each minute and start calculating a path with a Breadth first approach to find the fastest 
# way
wind_rounds = {}
for minute in range(12):
    set_minute = set()
    for wind_dir, positions in wind_dict.items():
        if (wind_dir == "<") or (wind_dir == ">"):
            move = np.array([0,1]) * minute
        else:
            move = np.array([1,0]) * minute

        if (wind_dir == "<") or (wind_dir == "^"):
            move = -1 * move

        for pos in positions:
            pos = np.array(pos)
            new_row, new_column = pos + move
            new_pos = (new_row % row_n, new_column % column_n)

            set_minute.add(new_pos)
    wind_rounds[minute] = set_minute


In [7]:
# For each round I want to find all unoccupied nodes, lets do some set operations!
nodes_set = set(nodes)

In [8]:
free_nodes = {}
for minute, wind_set in wind_rounds.items():
    free_nodes[minute] = nodes_set.difference(wind_set)

In [17]:
free_nodes

{0: {(0, 2), (1, 0), (1, 2), (1, 3), (2, 2)},
 1: {(0, 0),
  (0, 3),
  (0, 5),
  (1, 1),
  (1, 2),
  (1, 5),
  (2, 2),
  (2, 5),
  (3, 2),
  (3, 3)},
 2: {(0, 0),
  (0, 4),
  (0, 5),
  (1, 0),
  (2, 0),
  (2, 3),
  (3, 0),
  (3, 2),
  (3, 3),
  (3, 5)},
 3: {(0, 5),
  (1, 0),
  (1, 3),
  (1, 5),
  (2, 4),
  (2, 5),
  (3, 0),
  (3, 1),
  (3, 4),
  (3, 5)},
 4: {(0, 0),
  (0, 2),
  (0, 3),
  (1, 2),
  (1, 4),
  (1, 5),
  (2, 2),
  (2, 5),
  (3, 0),
  (3, 5)},
 5: {(0, 1),
  (0, 3),
  (1, 1),
  (1, 3),
  (1, 4),
  (2, 0),
  (3, 0),
  (3, 2),
  (3, 3),
  (3, 5)},
 6: {(0, 2), (0, 4), (1, 0), (2, 1), (2, 2), (3, 1), (3, 2), (3, 3), (3, 4)},
 7: {(0, 0), (0, 5), (1, 2), (1, 5), (2, 5), (3, 1), (3, 2), (3, 3), (3, 4)},
 8: {(0, 0),
  (0, 5),
  (1, 0),
  (1, 1),
  (1, 4),
  (2, 0),
  (2, 3),
  (2, 4),
  (3, 0),
  (3, 5)},
 9: {(0, 1), (0, 5), (1, 0), (1, 3), (1, 5), (2, 5), (3, 0), (3, 5)},
 10: {(0, 0),
  (0, 2),
  (0, 3),
  (1, 5),
  (2, 2),
  (2, 5),
  (3, 0),
  (3, 1),
  (3, 4),
  (3, 5)},

In [15]:
nodes_neigbors = {}
for i, (x,y) in enumerate(nodes):
    if x < row_n:
        nodes_neigbors[(x,y)] = [(x,y+1),(x+1,y)]
    else:
        nodes_neigbors[(x,y)] = [(x,y+1)]

In [16]:
nodes_neigbors

{(0, 0): [(0, 1), (1, 0)],
 (0, 1): [(0, 2), (1, 1)],
 (0, 2): [(0, 3), (1, 2)],
 (0, 3): [(0, 4), (1, 3)],
 (0, 4): [(0, 5), (1, 4)],
 (0, 5): [(0, 6), (1, 5)],
 (1, 0): [(1, 1), (2, 0)],
 (1, 1): [(1, 2), (2, 1)],
 (1, 2): [(1, 3), (2, 2)],
 (1, 3): [(1, 4), (2, 3)],
 (1, 4): [(1, 5), (2, 4)],
 (1, 5): [(1, 6), (2, 5)],
 (2, 0): [(2, 1), (3, 0)],
 (2, 1): [(2, 2), (3, 1)],
 (2, 2): [(2, 3), (3, 2)],
 (2, 3): [(2, 4), (3, 3)],
 (2, 4): [(2, 5), (3, 4)],
 (2, 5): [(2, 6), (3, 5)],
 (3, 0): [(3, 1), (4, 0)],
 (3, 1): [(3, 2), (4, 1)],
 (3, 2): [(3, 3), (4, 2)],
 (3, 3): [(3, 4), (4, 3)],
 (3, 4): [(3, 5), (4, 4)],
 (3, 5): [(3, 6), (4, 5)]}

In [37]:
def find_neighbours(node, row_limit, col_limit):
    row, col = node
    n = []
    if row + 1 < row_limit:
        n1 = (row + 1, col)
        n.append(n1)
    if row - 1 >= 0:
        n2 = (row - 1, col)
        n.append(n2)
    if col + 1 < col_limit:
        n3 = (row, col + 1)
        n.append(n3)
    if col - 1 >= 0:
        n4 = (row, col - 1)
        n.append(n4)
    return set(n)

In [38]:

minute = 0
paths = [(minute, None)]
stop = False
log = []
while stop == False:
    minute, cur_path = paths.pop(0)
    if cur_path == None:
        set_minute = minute % 700
        cur_set = free_nodes[set_minute]
        
        if (0,0) in cur_set:
            minute = minute + 1
            new_path = [minute, [(0,0)]]
            paths.append(new_path)
        else:
            minute = minute + 1
            new_path = [minute, None]
            paths.append(new_path)
            continue
    else:
        set_minute = minute % 700
        cur_set = free_nodes[set_minute]
        cur_pos = cur_path[-1]
        cur_neighbours = find_neighbours(cur_pos, row_limit = row_n, col_limit = column_n)
        pos_moves = cur_neighbours.intersection(cur_set)
        log.append((minute,cur_path, cur_pos, pos_moves))
        if len(pos_moves) != 0:
            for new_pos in pos_moves:
                new_path = cur_path.copy()
                cur_minute = minute + 1
                new_path.append(new_pos)
                new_path = (cur_minute, new_path)
                paths.append(new_path)
                if new_pos == (row_n-1,column_n-1):
                    stop = True
                    print(cur_minute, new_path)
        if len(pos_moves) == 0:
            if cur_pos not in cur_set:
                continue
            cur_minute = minute + 1
            new_path = cur_path.copy()
            new_path.append(cur_pos)
            new_path = (cur_minute, new_path)
            paths.append(new_path)

KeyboardInterrupt: 